In [1]:
import pandas as pd

# reduced data = 100 lines

In [2]:
DATASET_SIZE = 50
data = pd.read_csv('content/hindi_english_parallel.csv')
data

,hindi,english
0,अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें,Give your application an accessibility workout
1,एक्सेर्साइसर पहुंचनीयता अन्वेषक,Accerciser Accessibility Explorer
2,निचले पटल के लिए डिफोल्ट प्लग-इन खाका,The default plugin layout for the bottom panel
3,ऊपरी पटल के लिए डिफोल्ट प्लग-इन खाका,The default plugin layout for the top panel
4,उन प्लग-इनों की सूची जिन्हें डिफोल्ट रूप से नि...,A list of plugins that are disabled by default
...,...,...
4994,कृपया एंटर a संदेश.,Please enter a log message.
4995,कृपया एंटर लेखक से. नाम,Please enter the commit author 's name
4996,कृपया एंटर लेखक से. e.,Please enter the commit author 's e - mail add...
4997,कमिट,Commit


In [3]:
hindi = data['hindi'][:DATASET_SIZE]
english = data['english'][:DATASET_SIZE]

In [4]:
valid_idx = list(range(len(hindi)))
remove = []

In [5]:
for i in valid_idx:
  if type(hindi[i]) == float or len(hindi[i]) < 1:
    print(f"Caugt {i}")
    remove.append(i)
  if type(english[i]) == float or len(english[i]) < 1:
    print(f"Caugt {i}")
    remove.append(i)

In [6]:
valid_idx = list( set(valid_idx) - set(remove) )

# Tokenisation

In [7]:
from tqdm.autonotebook import tqdm

C:\Users\AMAN GUPTA\AppData\Local\Temp\ipykernel_12724\987820437.py:1: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [8]:
sentence_hindi_tokens = []
hindi_tokens = set()

for sentence in tqdm(hindi):
  if len(sentence) >= 1:
    sentence_hindi_tokens.append(sentence.split(' '))
    for token in sentence_hindi_tokens[-1]:
      hindi_tokens.add(token)

  0%|          | 0/50 [00:00<?, ?it/s]

In [9]:
sentence_english_tokens = []
english_tokens = set()

for sentence in tqdm(english):
  if len(sentence) >= 1:
    sentence_english_tokens.append(sentence.split(' '))
    for token in sentence_english_tokens[-1]:
      english_tokens.add(token)

  0%|          | 0/50 [00:00<?, ?it/s]

In [10]:
# english_tokens -= set([''])
# hindi_tokens -= set([''])

print(f"English Tokens = {len(english_tokens)}")
print(f'Hindi Tokens = {len(hindi_tokens)}')

English Tokens = 117
Hindi Tokens = 125


In [11]:
hindi_t2v = {}
hindi_v2t = {}
for i, tkn in tqdm(enumerate(hindi_tokens)):
  hindi_t2v[tkn] = i
  hindi_v2t[i] = tkn

0it [00:00, ?it/s]

In [12]:
english_t2v = {}
english_v2t = {}
for i, tkn in tqdm(enumerate(english_tokens)):
  english_t2v[tkn] = i
  english_v2t[i] = tkn

0it [00:00, ?it/s]

# Better Sentences for processing - Less ambiguous

In [13]:
import numpy as np

In [14]:
CONFIRM_MAP = {}
CONFIRM_IDX = []
confirm_set = set()

In [15]:
for i, tkn in enumerate(sentence_english_tokens):
  # print(f'IDX : {i} = {len(tkn)}')
  if len(tkn) == 1:
    CONFIRM_MAP[tkn[0]] = sentence_hindi_tokens[i][0]
    CONFIRM_IDX.append(i)
    confirm_set.add(tkn[0])
    confirm_set.add(sentence_hindi_tokens[i][0])
    # print( f'{tkn} -->> {sentence_hindi_tokens[i]}' )

CONFIRM_MAP

{'Method': 'विधि',
 'Property': 'गुणधर्म',
 'Value': 'मान',
 'Everything': 'सभी',
 'Source': 'स्रोत',
 'Description': 'वर्णन',
 'Show': 'दिखाएं',
 'ID': 'आईडी',
 'Toolkit': 'औजार',
 'Version': 'संस्करण',
 'Size': 'आकार',
 'WIDGET': 'विडजेट',
 'Layer': 'स्तर',
 'Alpha': 'अल्फा'}

# Generating More Confirm maps

In [16]:
from datetime import datetime, timedelta

TRAINING_TIME = 1 # minutes

In [17]:
start = datetime.now()

size = len(sentence_english_tokens)
epch = 1
while datetime.now() <= start + timedelta(minutes=TRAINING_TIME):
  idx1 = np.random.randint(low=0, high=size)
  idx2 = np.random.randint(low=0, high=size)

  diffset = set(sentence_english_tokens[idx1]) - set(sentence_english_tokens[idx2])
  if len(diffset) == 1:
    diff = set(sentence_hindi_tokens[idx1]) - set(sentence_hindi_tokens[idx2])
    if len(list(diff)) == 1:
      # print(1,list(diffset), list(cmn))
      CONFIRM_MAP[list(diffset)[0]] = list(diff)[0]
  elif len(diffset - confirm_set) == 1:
    diffset -= confirm_set
    diff = set(sentence_hindi_tokens[idx1]) - set(sentence_hindi_tokens[idx2])
    if len(list(diff)) == 1:
      # print(2, list(diffset), list(cmn))
      CONFIRM_MAP[list(diffset)[0]] = list(diff)[0]

  intersection = set(sentence_english_tokens[idx1]).intersection(set(sentence_english_tokens[idx2]))
  if len(list(intersection)) == 1:
    cmn = set(sentence_hindi_tokens[idx1]).intersection(set(sentence_hindi_tokens[idx2]))
    if len(list(cmn)) == 1:
      # print(3, list(intersection), list(cmn))
      CONFIRM_MAP[list(intersection)[0]] = list(cmn)[0]

In [18]:
CONFIRM_MAP

{'Method': 'विधि',
 'Property': 'गुणधर्म',
 'Value': 'मान',
 'Everything': 'सभी',
 'Source': 'स्रोत',
 'Description': 'वर्णन',
 'Show': 'दिखाएं',
 'ID': 'आईडी',
 'Toolkit': 'औजार',
 'Version': 'संस्करण',
 'Size': 'आकार',
 'WIDGET': 'विडजेट',
 'Layer': 'स्तर',
 'Alpha': 'अल्फा',
 'accessible': 'समय',
 '_': '',
 'of': 'की',
 'default': 'डिफोल्ट',
 'event': 'घटना',
 'application': 'अनुप्रयोग',
 'Accessible': 'पहुंचनीय',
 'fill': 'भराई',
 'and': 'और',
 'bottom': 'निचले',
 'Monitor': 'मानिटर',
 'top': 'ऊपरी',
 'Highlight': 'रकें',
 'Absolute': 'निरपेक्ष',
 'position': 'स्थिति',
 'Relative': 'सापेक्ष',
 'Event': 'घटना'}

# Probablity Matrix

In [19]:
app_mat = np.ones((len(english_tokens), len(hindi_tokens)))

In [20]:
for eng_i in tqdm(range(len(app_mat))):
  for hnd_j in range(len(app_mat[0])):
    # we are iterating over appearence matrix
    '''
    Now for each block we will look for frequency in dataset of hindi-english token pair
    '''
    # for eng_sentence in english:
    #   eng_count = eng_sentence.count(english_v2t[eng_i])
    #   for line_no, hindi_sentense in zip(valid_idx, hindi):
    #     hnd_count = hindi[line_no].count(hindi_v2t[hnd_j])

    for eng_sentence, hindi_sentense in zip(english, hindi):
      eng_count = eng_sentence.count(english_v2t[eng_i])
      hnd_count = hindi_sentense.count(hindi_v2t[hnd_j])

      app_mat[eng_i][hnd_j] += min(eng_count, hnd_count)

  0%|          | 0/117 [00:00<?, ?it/s]

In [21]:
app_mat

array([[2., 2., 1., ..., 1., 1., 1.],
       [2., 1., 1., ..., 1., 1., 1.],
       [2., 2., 1., ..., 1., 1., 1.],
       ...,
       [2., 1., 1., ..., 1., 1., 1.],
       [2., 1., 1., ..., 2., 1., 1.],
       [2., 1., 1., ..., 1., 1., 1.]])

In [22]:
app_mat[app_mat != 1] , len(app_mat[app_mat != 1]), len(app_mat.flatten())

(array([2., 2., 2., ..., 2., 2., 2.]), 1421, 14625)

In [23]:
def get_max_k(eng_word_idx, dictionary, k=1):
  print(english_v2t[eng_word_idx], end='\t\t')
  lst = [x for x in dictionary[eng_word_idx]]
  # print(lst)
  lst = np.array(lst)

  lst = lst.argsort()[-k:][::-1]
  print([hindi_v2t[x] for x in lst])

In [24]:
app_mat.shape

(117, 125)

# Forcefully assign confident map data to the probablity matrix

In [25]:
app_mat += 5

In [26]:
for key, value in CONFIRM_MAP.items():
  try :
    for i in range(app_mat.shape[1]):
      app_mat[english_t2v[key]][i] = 0
    for i in range(app_mat.shape[0]):
      app_mat[i][hindi_t2v[value]] = 0

    app_mat[english_t2v[key]][hindi_t2v[value]] = 1
  except Exception as e:
    print(e)

In [27]:
for i in range(117):
  get_max_k(i, app_mat, k=1)

Start		['करो/रोको']
action		['का']
Clear		['रोजनामचा']
description)		['को']
Selection		['_)']
they		['उन्हें']
lection		['_)']
Everything		['सभी']
Z		['एमडीआई-जेड-क्रम']
Console		['आईपाइथन']
ID		['आईडी']
recording		['करो/रोको']
entry		['हाइलाइट']
box		['आसंधि']
duration		['को']
currently		['लिए']
event		['लिए']
Accessibility		['से']
Selected		['चुने']
Layer		['स्तर']
Perform		['का']
WIDGET		['विडजेट']
/		['करो/रोको']
Browse		['जिसे']
plugin		['प्लग-इन']
Monitor		['मानिटर']
application		['अनुप्रयोग']
default		['डिफोल्ट']
Relative		['सापेक्ष']
plugins		['निष्क्रिय']
current		['से']
Highlight		['रकें']
methods		['जिसे']
stop		['करो/रोको']
from		['उन्हें']
sources		['उन्हें']
when		['आसंधि']
nodes		['आसंधि']
Event		['घटना']
bottom		['निचले']
and		['और']
private		['को']
attributes		['को']
monitor		['लिए']
_		['']
highlight		['हाइलाइट']
lear		['को']
border		['रंग']
of		['की']
Absolute		['निरपेक्ष']
last		['हाइलाइट']
console		['लिए']
Alpha		['अल्फा']
log		['रोजनामचा']
panel		['लिए']
opacity		

# Probablity Wise Assignment

In [28]:
def assign(app_mat):
  arg = app_mat.argmax()
  x = arg//app_mat.shape[1]
  y = arg%app_mat.shape[1]

  for i in range(app_mat.shape[1]):
    app_mat[x][i] = 0
  for i in range(app_mat.shape[0]):
    app_mat[i][y] = 0

  app_mat[x][y] = 1

In [29]:
i = 0
while True:
  assign(app_mat)
  ans = np.max(app_mat) == 1
  if ans == True:
    break
  i += 1
  print(i)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85


In [30]:
app_mat

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [31]:
dictionary = {}

for i in range(100):
  print(english_v2t[i], end='\t\t')
  print([hindi_v2t[ app_mat[i].argmax() ]])
  dictionary[english_v2t[i]] = hindi_v2t[ app_mat[i].argmax() ]

Start		['करो/रोको']
action		['करें']
Clear		['रोजनामचा']
description)		['नहीं']
Selection		['(C']
they		['घटित']
lection		['_)']
Everything		['सभी']
Z		['एमडीआई-जेड-क्रम']
Console		['आईपाइथन']
ID		['आईडी']
recording		['रेकोर्डिंग']
entry		['अंतिम']
box		['(नोड)']
duration		['अवधि']
currently		['लेने']
event		['']
Accessibility		['पहुंचनीयता']
Selected		['गए']
Layer		['स्तर']
Perform		['संपन्न']
WIDGET		['विडजेट']
/		['शुरू']
Browse		['एपीआई']
plugin		['प्लग-इन']
Monitor		['मानिटर']
application		['अनुप्रयोग']
default		['डिफोल्ट']
Relative		['सापेक्ष']
plugins		['रूप']
current		['इस']
Highlight		['रकें']
methods		['में']
stop		['करो']
from		['जैसे-जैसे']
sources		['उन्हें']
when		['आसंधि']
nodes		['बक्से']
Event		['घटना']
bottom		['निचले']
and		['और']
private		['छिपाएं']
attributes		['निजी']
monitor		['(बोर्डर)']
_		['']
highlight		['रंग']
lear		['हटाएं']
border		['सीमांत']
of		['की']
Absolute		['निरपेक्ष']
last		['प्रविष्ट']
console		['काम']
Alpha		['अल्फा']
log		['मिटाओ']
panel		['खाका

# Reordering the sentences.
## BiGram Model for 2 word sequence

In [32]:
def get2gram(sentence):
  sentence = sentence.split(" ")
  return [[x,y] for x,y in zip(sentence[:-1], sentence[1:])]

In [33]:
get2gram("Hi shay patric cormac")

[['Hi', 'shay'], ['shay', 'patric'], ['patric', 'cormac']]

# Get probablity for 1st token

In [34]:
start_token = {}
for i in hindi_tokens:
  start_token[i] = 0

for strt in sentence_hindi_tokens:
  start_token[strt[0]] += 1

start_token

{'': 0,
 'घटना': 4,
 'संस्करण': 1,
 'आकार': 1,
 'घटित': 0,
 'स्रोत': 1,
 'सापेक्ष': 1,
 'जैसे-जैसे': 0,
 'अल्फा': 1,
 'विडजेट': 1,
 'हाइलाइट': 2,
 'किए': 0,
 'एपीआई': 1,
 'करें': 0,
 'रूप': 0,
 'में': 0,
 'लेने': 0,
 'अपारदर्शिता।': 0,
 '(C': 0,
 'विधियों': 0,
 'उन्हें': 0,
 'संपन्न': 0,
 'ऊपरी': 1,
 'गए': 0,
 'स्तर': 1,
 'प्लग-इनों': 0,
 'और': 0,
 '(बोर्डर)': 0,
 'रोजनामचा': 0,
 'निचले': 1,
 'खाका': 0,
 'किया': 0,
 'अनुप्रयोग': 1,
 'जिन्हें': 0,
 'छिपाएं': 0,
 'रंग': 0,
 '_)': 0,
 'करो/रोको': 0,
 'उन': 1,
 'डिफोल्ट': 0,
 'दर्शाता': 0,
 '(नोड)': 0,
 'अवधि': 1,
 'गुणधर्म': 1,
 'पटल': 0,
 'घटनाएं': 0,
 'प्राप्त': 0,
 'एमडीआई-जेड-क्रम': 1,
 'नहीं': 0,
 'लाभ': 0,
 'मानिटर': 0,
 'विधि': 1,
 'विवरण': 0,
 'आसंधि': 0,
 'भराई': 1,
 'आईडी': 1,
 '(मेथड)': 0,
 'हटाएं': 0,
 'संग्रह': 1,
 'घटनाओं': 1,
 'की': 0,
 'औजार': 1,
 'जिसे': 0,
 'हो,': 0,
 'o)': 0,
 'आईपाइथन': 1,
 'स्थिति': 0,
 'पहुंचनीयता': 0,
 'प्लग-इन': 0,
 'बक्से': 0,
 '0,': 1,
 'चुनते': 0,
 'उसकी': 0,
 'काम': 0,
 'प्रकारों': 0,
 '(_': 0,

# Now we will get probablity of next word based on previous word

In [35]:
conditional_probablity = {}
for i in hindi_tokens:
  conditional_probablity[i] = {}
  for j in hindi_tokens:
    conditional_probablity[i][j] = 0

In [36]:
for first in conditional_probablity:
  for second in conditional_probablity[first]:

    for hnd_sen in hindi:
      search = first + " " + second
      conditional_probablity[first][second] += hnd_sen.count(search)

# Save the dictionary and probablity matrix

In [37]:
import json


with open('dict.json', 'w', encoding='utf-8') as outfile:
    json.dump(dictionary, outfile, ensure_ascii=False)

with open("joint_probab.json", 'w', encoding='utf-8') as outfile:
    json.dump(conditional_probablity, outfile, ensure_ascii=False)

with open('start_token.json', 'w', encoding='utf-8') as outfile:
    json.dump(start_token, outfile, ensure_ascii=False)
    

# Actual Translation and rearrangement

In [38]:
dictionary2 = None
with open('dict.json', 'r',encoding='utf-8') as f:
  dictionary2 = json.loads(f.read())

In [39]:
def dict_translator(eng_sen):
  tkns = eng_sen.split(" ")
  tkns = [dictionary2[x] for x in tkns]
  return " ".join(tkns), tkns

In [42]:
to_translate = "Show top types of Method"

In [43]:
intermediate, tkns = dict_translator(to_translate)
intermediate

'दिखाएं ऊपरी प्रकारों की में'

In [44]:
intermediate, tkns = dict_translator(to_translate)

def reorder(intermediate, tkns):
  rearranged = []
  start_probab = [start_token[x] for x in tkns]
  rearranged.append( tkns[ np.argmax(start_probab) ] )
  # rearranged, start_probab, tkns
  tkns.remove( rearranged[-1] )

  for _ in range(len(tkns)):
    next_probab = [conditional_probablity[rearranged[-1]][x] for x in tkns ]
    rearranged.append( tkns[ np.argmax(next_probab) ] )
    tkns.remove( rearranged[-1] )

  return ' '.join(rearranged)

reorder(intermediate, tkns)

'दिखाएं ऊपरी प्रकारों की में'

In [45]:
good_quality = "Method types bottom of Hide from Show Property Size top Selected Source opacity and plugin Monitor WIDGET position fill Relative action"

# Final Result

In [46]:
def full_translation(to_translate):
  intermediate, tkns = dict_translator(to_translate)
  return reorder(intermediate, tkns)

In [48]:
full_translation("Show top types of methods")

'दिखाएं ऊपरी प्रकारों की में'

In [49]:
full_translation('Hide Selected Monitor plugin')

'गुणों गए मानिटर प्लग-इन'

In [50]:
english

0        Give your application an accessibility workout
1                     Accerciser Accessibility Explorer
2        The default plugin layout for the bottom panel
3           The default plugin layout for the top panel
4        A list of plugins that are disabled by default
5                                    Highlight duration
6     The duration of the highlight box when selecti...
7                                Highlight border color
8        The color and opacity of the highlight border.
9                                  Highlight fill color
10         The color and opacity of the highlight fill.
11                                          API Browser
12    Browse the various methods of the current acce...
13                              Hide private attributes
14                                               Method
15                                             Property
16                                                Value
17                                      IPython 

In [80]:
full_translation('Relative position')

'सापेक्ष स्थिति'

In [81]:
hindi[43]

'सापेक्ष स्थिति'

# BLEU Score

In [83]:
input_string = "Hide private attributes"
input_string_hindi=["निजी गुणों को छिपाएं","भराई के रंग को हाइलाइट करें","सीमांत के रंग को हाइलाइट करें","अंतिम प्रविष्ट घटना को हाइलाइट करो","घटना रेकोर्डिंग शुरू करो/रोको","सापेक्ष स्थिति"]
predicted_hindi_string=["निजी गुणों छिपाएं","हाइलाइट रकें भराई","हाइलाइट रकें सीमांत","अंतिम प्रविष्ट  रकें","करो/रोको शुरू करो  रेकोर्डिंग","सापेक्ष स्थिति"]
token_array = []
token_array_hindi=[]
token_array_predicted_hindi=[]

In [85]:
# token_array = input_string.split()
original_hindi_array=[]
predicted_hindi_array=[]
for i in range(len(input_string_hindi)):
    token_array_hindi=input_string_hindi[i].split(' ')
    token_array_predicted_hindi=predicted_hindi_string[i].split(' ')
    original_hindi_array.append(token_array_hindi)
    predicted_hindi_array.append( token_array_predicted_hindi)
    


In [86]:
original_hindi_array

[['निजी', 'गुणों', 'को', 'छिपाएं'],
 ['भराई', 'के', 'रंग', 'को', 'हाइलाइट', 'करें'],
 ['सीमांत', 'के', 'रंग', 'को', 'हाइलाइट', 'करें'],
 ['अंतिम', 'प्रविष्ट', 'घटना', 'को', 'हाइलाइट', 'करो'],
 ['घटना', 'रेकोर्डिंग', 'शुरू', 'करो/रोको'],
 ['सापेक्ष', 'स्थिति']]

In [87]:
predicted_hindi_array

[['निजी', 'गुणों', 'छिपाएं'],
 ['हाइलाइट', 'रकें', 'भराई'],
 ['हाइलाइट', 'रकें', 'सीमांत'],
 ['अंतिम', 'प्रविष्ट', '', 'रकें'],
 ['करो/रोको', 'शुरू', 'करो', '', 'रेकोर्डिंग'],
 ['सापेक्ष', 'स्थिति']]

In [88]:
tot_original_len=[]
for i in range(len(predicted_hindi_array)):
    count=0
    for j in range(len(predicted_hindi_array[i])):
        count=count+1
    tot_original_len.append(count)
    
tot_original_len

[3, 3, 3, 4, 5, 2]

In [89]:
tot_predicted_len=[]
for i in range(len(predicted_hindi_array)):
    to_test_orignal_token_array_hindi=[x for x in original_hindi_array[i]]
    found_predicted_hindi_count=0
    for j in range(len(predicted_hindi_array[i])):
        for k in range(len(to_test_orignal_token_array_hindi)):
            if(predicted_hindi_array[i][j]==to_test_orignal_token_array_hindi[k]):
                found_predicted_hindi_count=found_predicted_hindi_count+1
#                 print(k)
#                 to_test_orignal_token_array_hindi.remove(to_test_orignal_token_array_hindi[k])
                
    tot_predicted_len.append(found_predicted_hindi_count)            
            
tot_predicted_len

[3, 2, 2, 2, 3, 2]

In [90]:
unicode_belu=[]
for i in range(len(predicted_hindi_array)):
    belu_score=(tot_predicted_len[i]/tot_original_len[i]) * 100
    unicode_belu.append(belu_score)
unicode_belu

[100.0, 66.66666666666666, 66.66666666666666, 50.0, 60.0, 100.0]

In [91]:
avg_belu=0
for i in range(len(unicode_belu)):
    avg_belu=avg_belu+unicode_belu[i]

overallbelu=avg_belu/len(unicode_belu)
overallbelu

73.88888888888889

# bigram

In [92]:
original_hindi_array

[['निजी', 'गुणों', 'को', 'छिपाएं'],
 ['भराई', 'के', 'रंग', 'को', 'हाइलाइट', 'करें'],
 ['सीमांत', 'के', 'रंग', 'को', 'हाइलाइट', 'करें'],
 ['अंतिम', 'प्रविष्ट', 'घटना', 'को', 'हाइलाइट', 'करो'],
 ['घटना', 'रेकोर्डिंग', 'शुरू', 'करो/रोको'],
 ['सापेक्ष', 'स्थिति']]

In [93]:
predicted_hindi_array

[['निजी', 'गुणों', 'छिपाएं'],
 ['हाइलाइट', 'रकें', 'भराई'],
 ['हाइलाइट', 'रकें', 'सीमांत'],
 ['अंतिम', 'प्रविष्ट', '', 'रकें'],
 ['करो/रोको', 'शुरू', 'करो', '', 'रेकोर्डिंग'],
 ['सापेक्ष', 'स्थिति']]

In [94]:
total_bigram_in_input_predicted=[]
for i in range(len(predicted_hindi_array)):
    count=0
    for j in range(len(predicted_hindi_array[i])):
        count=count+1
    total_bigram_in_input_predicted.append(count-1)
    
total_bigram_in_input_predicted

[2, 2, 2, 3, 4, 1]

In [95]:
total_bigram_in_orignal=[]
for i in range(len(original_hindi_array)):
    count=0
    for j in range(len(original_hindi_array[i])):
        count=count+1
    total_bigram_in_orignal.append(count-1)
    
total_bigram_in_orignal

[3, 5, 5, 5, 3, 1]

In [96]:
predicted_hindi_array

[['निजी', 'गुणों', 'छिपाएं'],
 ['हाइलाइट', 'रकें', 'भराई'],
 ['हाइलाइट', 'रकें', 'सीमांत'],
 ['अंतिम', 'प्रविष्ट', '', 'रकें'],
 ['करो/रोको', 'शुरू', 'करो', '', 'रेकोर्डिंग'],
 ['सापेक्ष', 'स्थिति']]

In [97]:
to_test_orignal_bigram_token_array_hindi=[x for x in original_hindi_array[0]]
to_test_orignal_bigram_token_array_hindi

['निजी', 'गुणों', 'को', 'छिपाएं']

In [98]:
predicted_hindi_array

[['निजी', 'गुणों', 'छिपाएं'],
 ['हाइलाइट', 'रकें', 'भराई'],
 ['हाइलाइट', 'रकें', 'सीमांत'],
 ['अंतिम', 'प्रविष्ट', '', 'रकें'],
 ['करो/रोको', 'शुरू', 'करो', '', 'रेकोर्डिंग'],
 ['सापेक्ष', 'स्थिति']]

In [99]:
original_hindi_array

[['निजी', 'गुणों', 'को', 'छिपाएं'],
 ['भराई', 'के', 'रंग', 'को', 'हाइलाइट', 'करें'],
 ['सीमांत', 'के', 'रंग', 'को', 'हाइलाइट', 'करें'],
 ['अंतिम', 'प्रविष्ट', 'घटना', 'को', 'हाइलाइट', 'करो'],
 ['घटना', 'रेकोर्डिंग', 'शुरू', 'करो/रोको'],
 ['सापेक्ष', 'स्थिति']]

In [100]:
bigram_output=[]
for i in range(len(predicted_hindi_array)):
    found_bigram_hindi_count=0;
    to_test_orignal_bigram_token_array_hindi=[x for x in original_hindi_array[i]]
    for j in range(total_bigram_in_input_predicted[i]):
        for k in range(total_bigram_in_orignal[i]):
            if(predicted_hindi_array[i][j]==to_test_orignal_bigram_token_array_hindi[k] and predicted_hindi_array[i][j+1]==to_test_orignal_bigram_token_array_hindi[k+1]):
                found_bigram_hindi_count=found_bigram_hindi_count+1
#                 to_test_orignal_bigram_token_array_hindi.remove(to_test_orignal_bigram_token_array_hindi[k])
    bigram_output.append(found_bigram_hindi_count)   
            
bigram_output

[1, 0, 0, 1, 0, 1]

In [101]:
bigram_belu_new=[]
for i in range(len(predicted_hindi_array)):
    belu_bigram_score=(bigram_output[i]/total_bigram_in_input_predicted[i]) * 100
    bigram_belu_new.append(belu_bigram_score)

bigram_belu_new

[50.0, 0.0, 0.0, 33.33333333333333, 0.0, 100.0]

In [102]:
final_bigram=0
for i in range(len(bigram_belu_new)):
    final_bigram=final_bigram+bigram_belu_new[i]

final_output=final_bigram/len(bigram_belu_new)
final_output

30.555555555555554